In [1]:
import pandas as pd
import re
import numpy as np
import requests
import urllib
import io

In [3]:
#insert crawl csv 
data = ('/Users/rutheverett/Downloads/www-deepcrawl-com_12-10-2020_All_Pages_basic.csv')
#update to include columns you would like to include from csv 
columns = ['url', 'level', 'http_status_code', 'indexable', 'links_in_count', 'followed_links_in_count', 'links_out_count', 'deeprank', 'backlink_count', 'backlink_domain_count']
df = pd.read_csv(data , usecols=columns)

In [7]:
#read first 10 rows
df.head(10)

,deeprank,level,url,http_status_code,indexable,links_in_count,links_out_count,followed_links_in_count,backlink_count,backlink_domain_count,segment
0,2.13,3,https://www.deepcrawl.com/knowledge/technical-...,200,True,3,88,3,NaN,NaN,Technical SEO Library
1,2.55,3,https://www.deepcrawl.com/knowledge/guides/sch...,200,True,6,114,6,NaN,NaN,Guides
2,2.19,3,https://www.deepcrawl.com/knowledge/technical-...,200,True,1,93,1,NaN,NaN,Technical SEO Library
3,2.51,3,https://www.deepcrawl.com/knowledge/guides/seg...,200,True,1,92,1,NaN,NaN,Guides
4,2.57,3,https://www.deepcrawl.com/blog/events/go-red-c...,200,True,13,108,13,NaN,NaN,Blog
5,2.07,3,https://www.deepcrawl.com/knowledge/technical-...,200,True,2,86,2,NaN,NaN,Technical SEO Library
6,2.07,3,https://www.deepcrawl.com/knowledge/technical-...,200,True,2,86,2,NaN,NaN,Technical SEO Library
7,2.59,3,https://www.deepcrawl.com/knowledge/technical-...,200,True,5,97,5,NaN,NaN,Technical SEO Library
8,2.55,3,https://www.deepcrawl.com/knowledge/guides/ren...,200,True,6,94,6,NaN,NaN,Guides
9,2.17,3,https://www.deepcrawl.com/knowledge/technical-...,200,True,2,100,2,NaN,NaN,Technical SEO Library


In [8]:
#set up segments 
segment_definitions = [
    [(r'\/blog\/'), 'Blog'],
    [(r'\/technical-seo-library\/'), 'Technical SEO Library'],
    [(r'\/hangout-library\/'), 'Hangout Library'],
    [(r'\/guides\/'), 'Guides'],
    [(r'\/case-studies\/'), 'Case Studies'],
    [(r'\/why-'), 'Solutions'],
    ]

use_segment_definitions = True

def get_segment(url):
    
    if use_segment_definitions == True:
        for segment_definition in segment_definitions:
            if re.findall(segment_definition[0], url):
                return segment_definition[1]
        return 'Other'

#apply segmentation to all URLs in dataframe 
df['segment'] = df['url'].apply(lambda x: get_segment(x))

In [10]:
#create pivot table to get a count of internal links to each segment 
total_internal_links = pd.pivot_table(df, index='segment', values=['url', 'links_in_count', 'followed_links_in_count', 'links_out_count'], aggfunc={'url':len, 'links_in_count':np.sum, 'followed_links_in_count':np.sum, 'links_out_count':np.sum})
total_internal_links

,followed_links_in_count,links_in_count,links_out_count,url
segment,,,,
Blog,19742,19742,77002,737
Case Studies,117,117,924,10
Guides,4127,4127,4967,67
Hangout Library,8019,8019,45261,406
Other,37050,37050,5498,88
Solutions,5143,5143,392,6
Technical SEO Library,2699,2699,2924,33


In [15]:
#create pivot table to get an of internal links to each segment 
average_internal_links = pd.pivot_table(df, index='segment', values=['url', 'links_in_count', 'followed_links_in_count', 'links_out_count'], aggfunc={'url':len, 'links_in_count':np.mean, 'followed_links_in_count':np.mean, 'links_out_count':np.mean})
average_internal_links['followed_links_in_count'] = (average_internal_links['followed_links_in_count']).apply('{:.1f}'.format)
average_internal_links['links_in_count'] = (average_internal_links['links_in_count']).apply('{:.1f}'.format)
average_internal_links['links_out_count'] = (average_internal_links['links_out_count']).apply('{:.1f}'.format)
average_internal_links

,followed_links_in_count,links_in_count,links_out_count,url
segment,,,,
Blog,26.8,26.8,104.5,737
Case Studies,11.7,11.7,92.4,10
Guides,61.6,61.6,74.1,67
Hangout Library,19.8,19.8,111.5,406
Other,421.0,421.0,62.5,88
Solutions,857.2,857.2,65.3,6
Technical SEO Library,81.8,81.8,88.6,33


In [17]:
#Additional step using Level and DeepRank to find averages for segments 
averages = pd.pivot_table(df, index='segment', values=['url', 'links_in_count', 'followed_links_in_count', 'links_out_count', 'deeprank', 'level'], aggfunc={'url':len, 'links_in_count':np.mean, 'followed_links_in_count':np.mean, 'links_out_count':np.mean, 'deeprank':np.mean, 'level':np.mean })
averages['followed_links_in_count'] = (averages['followed_links_in_count']).apply('{:.1f}'.format)
averages['links_in_count'] = (averages['links_in_count']).apply('{:.1f}'.format)
averages['links_out_count'] = (averages['links_out_count']).apply('{:.1f}'.format)
averages['deeprank'] = (averages['deeprank']).apply('{:.1f}'.format)
averages['level'] = (averages['level']).apply('{:.1f}'.format)
averages

,deeprank,followed_links_in_count,level,links_in_count,links_out_count,url
segment,,,,,,
Blog,1.7,26.8,4.9,26.8,104.5,737
Case Studies,3.9,11.7,2.2,11.7,92.4,10
Guides,1.8,61.6,3.8,61.6,74.1,67
Hangout Library,1.5,19.8,4.6,19.8,111.5,406
Other,2.9,421.0,3.5,421.0,62.5,88
Solutions,5.7,857.2,2.8,857.2,65.3,6
Technical SEO Library,2.3,81.8,3.2,81.8,88.6,33
